In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import quandl as qdl
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode,iplot

In [ ]:
api = qdl.ApiConfig.api_key = "Your Key"
start = '2000-1-1'
end = '2020-4-1'
msft=qdl.get('EOD/MSFT',start_date=start,end_date=end)
msftCopy = msft

In [ ]:
class BollingerBandsSimulator:

    def __init__(self, df_original, from_date, window_size, no_of_std, figsize=(20,10)):
        self.df = df_original
        self.from_date = from_date
        self.window_size = window_size
        self.no_of_std = no_of_std
        self.figsize = figsize
        self.sharpe_ratio = None
        self.returns = None
                
    def _build_bollinger_bands(self):
        rolling_mean = self.df['Close'].rolling(self.window_size).mean()
        rolling_std = self.df['Close'].rolling(self.window_size).std()

        self.df['Rolling Mean'] = rolling_mean
        self.df['Bollinger High'] = rolling_mean + (rolling_std * self.no_of_std)
        self.df['Bollinger Low'] = rolling_mean - (rolling_std * self.no_of_std)

    def _calculate_positions(self):
            # create a new column in the DataFrame to hold signal information
            self.df['Signal'] = None
            self.df['Position'] = None
            mode = 'neutral'
            self.df['R'] = None
            for index in range(len(msft)):
                if index == 0:
                    continue
                row = self.df.iloc[index]
                prev_row = self.df.iloc[index - 1]
                self.df.iloc[index,self.df.columns.get_loc('R')] = (row['Close']-prev_row['Close'])/prev_row['Close']
                
            for index in range(len(msft)):
                if index == 0:
                    continue

                row = self.df.iloc[index]
                prev_row = self.df.iloc[index - 1]
                ## Entering a position

                # long
                if mode == 'neutral' and row['R'] < 0:
                    self.df.iloc[index, self.df.columns.get_loc('Signal')] = 1
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 1
                    mode = 'long'

                # short
                elif mode == 'neutral' and row['R'] > 0:
                    self.df.iloc[index, self.df.columns.get_loc('Signal')] = -1
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
                    mode = 'short'

                ## Exiting a position
                elif mode == 'long' and row['R'] > 0:
                    self.df.iloc[index, self.df.columns.get_loc('Signal')] = -1
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0
                    mode = 'neutral'

                elif mode == 'short' and row['R'] < 0:
                    self.df.iloc[index, self.df.columns.get_loc('Signal')] = 1
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0
                    mode = 'neutral'    

                else:
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = self.df.iloc[index-1, self.df.columns.get_loc('Position')]        



    def _calculate_returns(self):
        self.df['Returns']=self.df['Signal']**2*self.df['Close']
        self.df['Returns'].fillna(method='ffill', inplace=True)
        self.df['Returns']=self.df['Returns'].pct_change()*self.df['Position'].shift()
        self.df['Returns']=self.df['Returns'].fillna(value=0)
        self.returns = self.df['Returns'].dropna()
        # Removing all the zero returns i.e. where we never traded
        self.returns=self.returns[self.returns!=0]
        self.sharpe_ratio = self.returns.mean()/(self.returns.var())**(1/2)
        

    def _plot_returns(self):
        self.df['Returns'].cumsum().plot(figsize=self.figsize)

    def simulate(self):
        self._build_bollinger_bands()
        self._calculate_positions()
        self._calculate_returns()
        self._plot_returns()

        return (
            self.window_size, 
            self.no_of_std, 
            self.sharpe_ratio
        )

In [ ]:
simulator = BollingerBandsSimulator(
    df_original=msft, 
    from_date="2002-01-01", 
    window_size=10, 
    no_of_std=1.5
)
simulator.simulate()
# plt.savefig("Returns_cumsum_2.png")

# The graph shows that we have a net return of 60% in 5 years

In [ ]:
msft['Returns'].plot(figsize=(20,10))
# plt.savefig("Returns2.png")

In [ ]:
msft['Returns_cumsum'] = msft['Returns'].cumsum()

In [ ]:
msft